In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.legend_handler import HandlerLine2D

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import roc_curve, auc, roc_curve
from sklearn.metrics import confusion_matrix, roc_auc_score, recall_score, precision_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from gensim.models import Word2Vec, Phrases
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

import pickle
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

import re
%matplotlib inline

Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [0]:
cv = pickle.load(open('./drive/My Drive/data/cv.pkl', 'rb'))

In [4]:
input_length = 150
trigrams_model = Word2Vec.load('./drive/My Drive/data/dv/trigram.model')
bigrams = Phrases.load('./drive/My Drive/data/bi.model')
trigrams = Phrases.load('./drive/My Drive/data/tri.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    text = text.lower()
    return re.sub(r'[^a-z]',' ',text)

def lemmatize(tokens):
    tokens = list(map(lemmatizer.lemmatize, tokens))
    lemmatized_tokens = list(map(lambda x: lemmatizer.lemmatize(x, 'v'), tokens))
    return ' '.join(list(filter(lambda x: not x in stop_words, lemmatized_tokens)))

def preprocess(text):
    text = clean_text(text)
    tokens = word_tokenize(text)
    return lemmatize(tokens)
    
def vectorize_data(data, vocab):
    keys = list(vocab.keys())
    filter_unknown = lambda word: vocab.get(word, None) is not None
    encode = lambda review: list(map(keys.index, filter(filter_unknown, review)))
    vectorized = list(map(encode, data))
    return vectorized

In [6]:
model = load_model('./drive/My Drive/data/bilstm_2.h5')
lr = pickle.load(open('./drive/My Drive/data/lr.pkl','rb'))
mnb = pickle.load(open('./drive/My Drive/data/mnb.pkl','rb'))
svm = pickle.load(open('./drive/My Drive/data/svm.pkl','rb'))





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [0]:
def infer(x):
  x = preprocess(x)
  test = cv.transform([x])
  sentiment = 'Positive' if lr.predict(test) == [1.] else 'Negative'
  print('LR: ' + sentiment)
  sentiment = 'Positive' if mnb.predict(test) == [1.] else 'Negative'
  print('MNB: ' + sentiment)
  sentiment = 'Positive' if svm.predict(test) == [1.] else 'Negative'
  print('SVM: ' + sentiment)
  data = vectorize_data(trigrams[bigrams[np.array([x.split()])]], vocab=trigrams_model.wv.vocab)
  X = pad_sequences(
      sequences=data,
      maxlen=input_length,
      padding='post')
  sentiment = 'Positive' if model.predict_classes(X) == [[1]] else 'Negative'
  print('BiLSTM: ' + sentiment)

In [15]:
x = 'I do  think this is bad to stay here'
print(infer(x))

LR: Positive
MNB: Negative
SVM: Negative
BiLSTM: Negative
None


/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
